# 1. Story generation with simple agent


In [1]:
%%capture --no-stderr
# %pip install "pyautogen>=0.2.26"

In [2]:
%%capture --no-stderr
# %pip install "deepeval>=0.21.33"

In [3]:
%%capture --no-stderr
# %pip install "cohere"

## Execution parameters

In [4]:
#define the start message
start_message = """
    Create for me a story for a ten panels sci-fi comic, the story must have at most 5 characters.
    Your response must contain only the story and no other text
"""

In [5]:
#set here the API Keys used by deepeval (autogen uses configurations in OAI_CONFIG_LIST file
import os
os.environ["OPENAI_API_KEY"] = "<your_api_key>"
os.environ["COHERE_API_KEY"] = "<your_api_key>"

In [6]:
#set the seed
seed = 42

In [7]:
#select which llm models you want to use for comic generation
enabled_models = [
    "gpt-3.5-turbo",
    "gpt-4",
    "command-nightly",
    "command-r",
]  

In [8]:
#select which llm models you want to use for output evaluation
enabled_evaluation_models = [
    "gpt-3.5-turbo",
    "gpt-4",
    "command-nightly",
    "command-r",
]

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [9]:
#create llm_configs list
import autogen

config_lists = {
    "command-nightly": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={
            "model": ["command-nightly"],
        },
    ),
    "command-r": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={
            "model": ["command-r"],
        },
     ),
    "gpt-3.5-turbo": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={
            "model": ["gpt-3.5-turbo"],
        },
    ),
    "gpt-4": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={
            "model": ["gpt-4"],
        },
    ),
    "mistral-7B": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={
            "model": ["mistral-7B"],
        },
    ),
}

llm_configs = []
for enabled_model in enabled_models:
    llm_configs.append({"config_list": config_lists[enabled_model], "cache_seed": seed})

## Import Libraries

In [10]:
from autogen import Agent, AssistantAgent, UserProxyAgent

## Define Agents - User Proxy
The UserProxyAgent is conceptually a proxy agent for humans

In [11]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    #max_consecutive_auto_reply=1, #Not used if we set human_input_mode to "ALWAYS" 
    default_auto_reply=None, #No auto reply
    llm_config=False, #No llm reply
    #system_message="A human admin.", #Only used when llm_config is not False
    code_execution_config=False, #No code execution is needed
    description="Simple human agent",
)


## Define Agents - Assistant
The AssistantAgent is designed to act as an AI assistant, using LLMs by default but not requiring human input or code execution

In [12]:
# Sistem message for the assistant
system_message= """
    As a comic story maker in this position, you must possess strong collaboration and communication abilities to efficiently complete tasks assigned
    by leaders or colleagues within a group chat environment. You create stories with the aim of creating a new original comic.
    Your responses MUST ALWAYS include a full story version with all the panels.
    If you receive a number of panels to be made, RESPECT IT.
    The story must contain full dialogues to be reported in the comic.
    For every panel provide two sections, an image description and the full dialogues to fit in. Dialogues must be short.
    Your responses MUST contains ONLY the story with NO other texts, write the story in the following format:

    TITLE: the story title
    ABSTRACT: short story summary

    CHARACTERS: names and short descritpions of the characters

    PANEL START progressive panel number
    IMAGE_DESCRIPTION: the panel image description
    IMAGE_DIALOGUES: the panel dialogues specifying the character who says them
    PANEL END progressive panel number
"""

In [13]:
# Assistant Agent definitions
assistants = []
for llm_config in llm_configs:
    assistants.append(AssistantAgent(
        name="assistant",
        system_message=system_message,
        llm_config=llm_config, #An llm configuration
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
        max_consecutive_auto_reply=1, #No consecutive reply
        description="Simple llm agent",
    ))


## Start the chats
Start

In [14]:
#Get last story produced:
def extract_story(agent: Agent) -> str:
    """
    Extracts the story from the last message of an agent.
    """
    # Function implementation...
    story = agent.last_message()["content"]
    return story

In [15]:
# Start the chats and extract stories
stories = []
for assistant in assistants:
    print("==============================")
    print("Starting Chat using model: ", assistant.llm_config['config_list'][0]['model'])
    print("==============================")
    # reset the assistant. Always reset the agents before starting a new conversation.
    assistant.reset()
    user_proxy.reset()
    user_proxy.initiate_chat(
        assistant,
        message=start_message,
    )
    stories.append(extract_story(assistant))
    print("==============================")
    print("Chat Ends")
    print("==============================")

Starting Chat using model:  gpt-3.5-turbo
user_proxy (to assistant):


    Create for me a story for a ten panels sci-fi comic, the story must have at most 5 characters.
    Your response must contain only the story and no other text


--------------------------------------------------------------------------------
assistant (to user_proxy):


    TITLE: The Alien Alliance
    ABSTRACT: In a distant galaxy, a group of humans and aliens must team up to save their worlds from a common threat.

    CHARACTERS:
    1. Captain Jaxar - A fearless human spaceship captain.
    2. Zara - A skilled alien engineer with a mysterious past.
    3. Krognar - A grumpy but brilliant alien scientist.
    4. Lieutenant Mia - Jaxar's loyal and resourceful second-in-command.
    5. Xel'tar - A charming alien trader with a shady side.

    PANEL START 1
    IMAGE_DESCRIPTION: The crew of the spaceship "Stellar Horizon" gathered around a holographic map showing multiple planets under attack.
    IMAGE_DIALOG

## Evaluate the results
Evaluation

## Define Agents - Assistant
Now we define an AI assistant, we are using LLMs to evaluate LLM results 

In [16]:
# import deepeval and dependencies
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval
from langchain_cohere import ChatCohere
# from langchain_community.chat_models import ChatCohere #deprecated
from deepeval.models.base_model import DeepEvalBaseLLM

In [17]:
# disable warnings to silent deepeval ipywidgets check
import warnings
warnings.filterwarnings('ignore')

In [18]:
#Define a custom evaluation model class (using Cohere command-nightly or command-r)

from langchain_community.chat_models import ChatCohere
from deepeval.models.base_model import DeepEvalBaseLLM

class Cohere(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Cohere Model"


In [19]:
# define here instances of llm model used by deepeval for evaluation
evaluation_models = {
    "gpt-3.5-turbo": "gpt-3.5-turbo",
    "gpt-4": "gpt-4",
    "command-nightly": Cohere(ChatCohere(model="command-nightly", seed=seed)),
    "command-r":       Cohere(ChatCohere(model="command-r", seed=seed)),
}

In [20]:
for enabled_evaluation_model in enabled_evaluation_models:
    eval_model_instance = evaluation_models[enabled_evaluation_model]
    for provided_output, enabled_model in zip(stories, enabled_models):
        print("\n==============================")
        print(f"Using evaluating model: {enabled_evaluation_model} to evaluate output from LLM: {enabled_model}")
        test_case = LLMTestCase(input=(system_message+start_message), actual_output=provided_output)
        coherence_metric = GEval(
            model=eval_model_instance,  # API usage
            name="Comic evaluation",
            # NOTE: you can only provide either criteria or evaluation_steps, and not both
            #criteria="Comic evaluation - the collective quality of comic panels, characters and images descriptions",
            evaluation_steps=[
                "Check whether the output format in 'actual output' aligns with that required in 'input'",
                "Check whether the sentences in 'actual output' aligns with that in 'input'",
                "Evaluate the general quality of comic panels in 'actual output'",
                "Evaluate the general quality of comic story in 'actual output'",
                "Evaluate the general quality of comic dialogues in 'actual output'",
                "Evaluate the general quality of characters descriptions in 'actual output'",
                "Evaluate the general quality of images descriptions in 'actual output'",
                "Be critical and emphasize the negative aspects of your evaluation",
            ],
            evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
        )
    
        coherence_metric.measure(test_case)
        print(f" Score: {coherence_metric.score}")
        print(f"Reason: {coherence_metric.reason}")
        print("==============================")


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Using evaluating model: gpt-3.5-turbo to evaluate output from LLM: gpt-3.5-turbo


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

...

 Score: 0.8
Reason: The 'actual output' mostly adheres to the format and content specified in the 'input' requirements, with a few minor deviations. The story is engaging and the panels build suspense effectively. However, the dialogue could be more varied and dynamic, with some sentences being overly expository. The character descriptions are concise but could be more creative, and the image descriptions could provide more detail to enhance the visual impact.

Using evaluating model: command-r to evaluate output from LLM: command-nightly


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

 Score: 0.9
Reason: The output follows the required format and includes all necessary sections. The story is engaging and well-structured, with an interesting premise and clear character roles. The panels build suspense and the dialogues are concise and effective. However, the negative aspects include the lack of a distinct ending and the need for slightly more detailed image descriptions in some panels.

Using evaluating model: command-r to evaluate output from LLM: command-r


 Score: 0.9
Reason: The story structure and format are well-aligned with the given criteria, and the narrative is engaging with clear, concise dialogues. The panels effectively build tension and showcase the robotic rebellion. However, there is room for improvement in the character development, as the story could delve deeper into their motivations and backgrounds, especially for the secondary characters like The Commander and Officer X7.
